# Data Cleaning

In this notebook I inspect the data to make sure it is clean. I also preprocess the data to make it easier to visualize. I also create some complex features to pass to the exploratory data analysis stage. 

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

np.random.seed(42)

import warnings
warnings.filterwarnings("ignore")

In [3]:
# load data
df = pd.read_csv("kc_house_data.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# 1.0 Cleaning

First I will make sure the data is clean by checking that all of the data types are appopriate, That there aren't any unecessary columns, There are no null values, and that all the values make sense.

## 1.1 Checking Data Types

In [4]:
# print out all of the column names with there data type
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

Zipcode is currently a numerical data type. This doesn't make since because there is no way to gain insight out of the zipcode's value with arithmatic. So I will change the zipcode column to the string data type to better represent that its a categorical value.

In [5]:
# change zipcode's data type to string
df["zipcode"] = df["zipcode"].astype(str)

## 1.2 Remove Unecessary values

By looking at the data I can see that the "id" and "date" columns are irrelevant to the problem so I will drop them.

In [6]:
# drop id and date from the data
df.drop(["id", "date"], axis=1, inplace=True)

## 1.3 Check For Nulls and Duplicates

In [7]:
# check for null values
df.isna().sum().sum()

0

In [10]:
# check for duplicate values
df.duplicated().sum()

5

In [11]:
# remove duplicate values
df = df.drop_duplicates()

## 1.4 Make Sure All Values Make Sense

I will quickly check that the values make sense by checking there minimum and maximum values.

In [12]:
# check minimum and maximum values
pd.concat([df.min(), df.max()], axis=1)

,0,1
price,75000.0000,7.700000e+06
bedrooms,0.0000,3.300000e+01
bathrooms,0.0000,8.000000e+00
sqft_living,290.0000,1.354000e+04
sqft_lot,520.0000,1.651359e+06
floors,1.0000,3.500000e+00
waterfront,0.0000,1.000000e+00
view,0.0000,4.000000e+00
condition,1.0000,5.000000e+00
grade,1.0000,1.300000e+01


There are some high values that may be outliers, but the values that seem illogical are the zero values for bedrooms and bathrooms. Now I will take a deeper look into what proportion of each column is filled with zeros.

In [14]:
# create function to print out % zeros for each column
def count_zeros(df, hide_cols=True):
    """
    function to print out % zeros for each column of a dataframe
    """
    
    # create a list for storing results
    results = []
    
    # loop over all columns
    for col in df:
        
        # find the amount of zeros with conditional slicing
        zeros = df[df[col] == 0][col].count()
    
        # divide the count of zeros by the length of the dataframe
        # multiply by 100 to get a percent value
        proportion = (zeros / len(df)) * 100
    
        # use a conditional to only save columns that contain a
        # zero value
        if proportion > 0:
            
            # store column name and proportion of zeros in "results"
            results.append([col, proportion])
            
    # take the results and put them in a dataframe for easy viewing
    return pd.DataFrame(results, columns=["column", "% zeros"])

# call function
count_zeros(df)

,column,% zeros
0,bedrooms,0.060163
1,bathrooms,0.046279
2,waterfront,99.245650
3,view,90.174935
4,sqft_basement,60.732136
5,yr_renovated,95.774713


Only a small proportion of values for bedrooms and bathrooms are zero so I will just remove those rows from the data.

In [15]:
# remove rows with a zero value for bedrooms or bathrooms
df = df[df["bedrooms"] != 0]
df = df[df["bathrooms"] != 0]